In [47]:
from shapely.geometry import Point
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

##### Read Raw Files of Covid Cases and Vaccines

In [48]:
covid_df = pd.read_csv('../data/COVID_CountByZipPerDate 03292021.csv')
vaccine_df = pd.read_csv('../data/vaccine_with_protocol.csv')

In [49]:
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8832 entries, 0 to 8831
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Specimen Collection Date  8832 non-null   object 
 1   Zip                       8456 non-null   float64
 2   Cases                     8832 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 207.1+ KB


In [50]:
vaccine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6694 entries, 0 to 6693
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   vacc_date    6694 non-null   object 
 1   zipcode      6462 non-null   float64
 2   count_given  6686 non-null   float64
 3   dose_nbr     6686 non-null   float64
 4   protocol     6694 non-null   object 
dtypes: float64(3), object(2)
memory usage: 261.6+ KB


##### Rename the Column Headers

In [51]:
vaccine_df = vaccine_df.rename(columns={'vacc_date':'date','zipcode':'zip','count_given':'vaccines','dose_nbr':'dose'})
covid_df =covid_df.rename(columns={'Specimen Collection Date':'date','Zip':'zip','Cases':'cases'})

##### Change of Data Types
The Covid Zipcodes are changed from float64 to String in Covid_df
The Vaccine Zipcode are changed from float64 to String   
The dates are changed to date type   

In [52]:
covid_df['zip'] = covid_df['zip'].astype('Int64').astype(str)
covid_df['date']= pd.to_datetime(covid_df['date'])
vaccine_df['zip'] = vaccine_df['zip'].astype('Int64').astype(str)
vaccine_df['date'] = pd.to_datetime(vaccine_df['date'])

In [53]:
covid_df.info()
print('\n***********************************************************\n')
vaccine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8832 entries, 0 to 8831
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    8832 non-null   datetime64[ns]
 1   zip     8832 non-null   object        
 2   cases   8832 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 207.1+ KB

***********************************************************

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6694 entries, 0 to 6693
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      6694 non-null   datetime64[ns]
 1   zip       6694 non-null   object        
 2   vaccines  6686 non-null   float64       
 3   dose      6686 non-null   float64       
 4   protocol  6694 non-null   object        
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 261.6+ KB


#### Drop Rows that do not have ZIP numbers in the Vaccine_df

In [54]:
vaccine_df.info()
vaccine_df.dropna(inplace=True)
print('After Dropping \n\n')
vaccine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6694 entries, 0 to 6693
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      6694 non-null   datetime64[ns]
 1   zip       6694 non-null   object        
 2   vaccines  6686 non-null   float64       
 3   dose      6686 non-null   float64       
 4   protocol  6694 non-null   object        
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 261.6+ KB
After Dropping 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6686 entries, 0 to 6693
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      6686 non-null   datetime64[ns]
 1   zip       6686 non-null   object        
 2   vaccines  6686 non-null   float64       
 3   dose      6686 non-null   float64       
 4   protocol  6686 non-null   object        
dtypes: datetime64[ns](1), float64(2), object(2)
memo

##### Clean up the count of Vaccines
The team has decided the one vaccine is considered an individual to be vaccinated

In [55]:
vac_clean_df=vaccine_df[vaccine_df['dose']==1]
print('vaccine_df sum -',vaccine_df['vaccines'].sum())
print('vac_clean sum -',vac_clean_df['vaccines'].sum())

vaccine_df sum - 334890.0
vac_clean sum - 220038.0


###### Replace the Data Frame of vaccination cleaned to the original df

In [56]:
vaccine_df=vac_clean_df
vaccine_df

,date,zip,vaccines,dose,protocol
0,2020-12-12,37215,1.0,1.0,two-shot
2,2020-12-14,37215,1.0,1.0,two-shot
3,2020-12-15,37221,1.0,1.0,two-shot
4,2020-12-16,37215,1.0,1.0,two-shot
5,2020-12-17,37013,16.0,1.0,two-shot
...,...,...,...,...,...
6686,2021-04-07,37221,81.0,1.0,two-shot
6687,2021-04-07,37221,2.0,1.0,one-shot
6689,2021-04-07,37228,9.0,1.0,two-shot
6691,2021-04-07,<NA>,8.0,1.0,two-shot


#### Read Ethnicity Population and Gather Total Population Figures to % cases and % vaccines

In [57]:
# Read the data file in Excel into the application, Skipping the header and loading sheet 0 is actual value sheet 1 = %
pop_df = pd.read_excel('..\data\hispanic\originbyrace.xlsx', skiprows = 1, header=[0, 1], sheet_name = 0)

###### Clean the pop_df file to extract only total population

In [58]:

#Drop columns with Error next to each of the columns
pop_df = pop_df[pop_df.columns.drop(list(pop_df.filter(regex='Error')))]
#Drop a Level of 'Value' headers that mean nothing to the data
pop_df.columns = pop_df.columns.droplevel(1)
# Remove unnecessary rows that will not be part of our statistics or you can choose to select certain rows into a DF
# Two separate data frame created. One of only the total population and the other to represent different races
#ethnicity_df = pop_df.drop([0,1,9,10,12,13,14,15,16,17,18,19,20])
pop_df = pop_df.iloc[[0],:]

# #Rotate the dataframe so that Zip can be column and the race data set across
pop_df= pop_df.set_index('Unnamed: 0_level_0').T.reset_index()
#ethnicity_df= ethnicity_df.set_index('Unnamed: 0_level_0').T.reset_index()
# #Rename the index row to Zip
pop_df = pop_df.rename(columns={'index':'zip','Total:':'population'})
#ethnicity_df=ethnicity_df.rename(columns={'index':'zip','Total:':'total','Two or more races:':'Two or more races',
#                                        'Hispanic or Latino:':'Hispanic or Latino'})

pop_df

Unnamed: 0_level_0,zip,population
0,37010,4966
1,37012,2185
2,37013,97939
3,37014,2835
4,37015,17665
...,...,...
113,38552,157
114,38560,1405
115,38563,3410
116,38567,685


In [59]:
# Read the data file in Excel into the application, Skipping the header and loading sheet 0 is actual value sheet 1 = %
ethnicity_df = pd.read_excel('..\data\hispanic\originbyrace.xlsx', skiprows = 1, header=[0, 1], sheet_name = 1)
#ethnicity_df

### Merge Population by Zip to Covid Cases & Vaccine Cases as a %
    - First summarize the cases and vaccines by Zip 
    - Merge population dataframe to each of the dataframes
    

#### Create dataframe for covid by zip

In [60]:
#cases per zipcode
covid_by_zip =covid_df.groupby('zip',as_index=False)['cases'].sum()
#sort by case Number
#covid_by_zip  =covid_by_zip.sort_values('cases', ascending=False)
#Remove rows that have <NA.
covid_by_zip.drop(index=[31], axis=0, inplace=True)
covid_by_zip

,zip,cases
0,37013,13354
1,37027,231
2,37069,16
3,37072,2984
4,37076,4279
5,37080,574
6,37115,4574
7,37138,2359
8,37189,479
9,37201,299


##### Create dataframe for vaccines by zip

In [61]:
#vaccines per zipcode
vac_by_zip =vaccine_df.groupby('zip',as_index=False)['vaccines'].sum()
vac_by_zip.vaccines.sum()
#vaccines by zip
vac_by_zip =vac_by_zip.sort_values('vaccines', ascending=False)
#Remove rows that have <NA.
vac_by_zip.drop(index=[31], axis=0, inplace=True)
vac_by_zip

,zip,vaccines
18,37211,19175.0
0,37013,18465.0
28,37221,17222.0
12,37205,14666.0
16,37209,13600.0
22,37215,12132.0
4,37076,11113.0
13,37206,10124.0
14,37207,9255.0
21,37214,9198.0


#### Identify the Largest and Smallest Covid and Vaccine Numbers

In [62]:
top5cases = covid_by_zip.nlargest(5,'cases')
top5vaccines = vac_by_zip.nlargest(5,'vaccines')
bottom5cases = covid_by_zip.nsmallest(5,'cases')
bottom5vaccines = vac_by_zip.nsmallest(5,'vaccines')

print('The Top Covid Cases by Zip\n\n',top5cases)
print('\n***********************************\n')
print('The Bottom Covid Cases by Zip\n\n',bottom5cases)
print('\n***********************************\n')
print('The Top Vaccine Numbers by Zip\n\n',top5vaccines)
print('\n***********************************\n')
print('The Bottom Vaccine Numbers by Zip\n\n',bottom5vaccines)

The Top Covid Cases by Zip

       zip  cases
0   37013  13354
18  37211  10233
16  37209   5072
14  37207   4729
6   37115   4574

***********************************

The Bottom Covid Cases by Zip

       zip  cases
30  37238      3
2   37069     16
20  37213     65
1   37027    231
26  37219    270

***********************************

The Top Vaccine Numbers by Zip

       zip  vaccines
18  37211   19175.0
0   37013   18465.0
28  37221   17222.0
12  37205   14666.0
16  37209   13600.0

***********************************

The Bottom Vaccine Numbers by Zip

       zip  vaccines
30  37238       3.0
2   37069      50.0
20  37213      72.0
1   37027     352.0
9   37201     576.0


###### Merge Covid and Vaccine Dataframes as Zip is the same for both data sets

In [63]:
cov_plus_vac = pd.merge(left=covid_by_zip, right=vac_by_zip, how='left', left_on='zip', right_on='zip')
cov_plus_vac

,zip,cases,vaccines
0,37013,13354,18465.0
1,37027,231,352.0
2,37069,16,50.0
3,37072,2984,8460.0
4,37076,4279,11113.0
5,37080,574,1970.0
6,37115,4574,8520.0
7,37138,2359,7152.0
8,37189,479,1056.0
9,37201,299,576.0


##### Bring Total Population by Zip into the cov_plus_vac dataframe 

In [64]:
c_v_zip= pd.merge(left=cov_plus_vac, right=pop_df, how='left', left_on='zip', right_on='zip')
c_v_zip

,zip,cases,vaccines,population
0,37013,13354,18465.0,97939.0
1,37027,231,352.0,56117.0
2,37069,16,50.0,20467.0
3,37072,2984,8460.0,31492.0
4,37076,4279,11113.0,40161.0
5,37080,574,1970.0,7394.0
6,37115,4574,8520.0,40343.0
7,37138,2359,7152.0,23784.0
8,37189,479,1056.0,4016.0
9,37201,299,576.0,1619.0


In [65]:
top5cases = c_v_zip.nlargest(5,'cases')
top5vaccines = c_v_zip.nlargest(5,'vaccines')
bottom5cases = c_v_zip.nsmallest(5,'cases')
bottom5vaccines = c_v_zip.nsmallest(5,'vaccines')

print('The Top Covid Cases by Zip\n\n',top5cases)
print('\n***********************************\n')
print('The Bottom Covid Cases by Zip\n\n',bottom5cases)
print('\n***********************************\n')
print('The Top Vaccine Numbers by Zip\n\n',top5vaccines)
print('\n***********************************\n')
print('The Bottom Vaccine Numbers by Zip\n\n',bottom5vaccines)

The Top Covid Cases by Zip

       zip  cases  vaccines  population
0   37013  13354   18465.0     97939.0
18  37211  10233   19175.0     75955.0
16  37209   5072   13600.0     38664.0
14  37207   4729    9255.0     39544.0
6   37115   4574    8520.0     40343.0

***********************************

The Bottom Covid Cases by Zip

       zip  cases  vaccines  population
30  37238      3       3.0         NaN
2   37069     16      50.0     20467.0
20  37213     65      72.0        95.0
1   37027    231     352.0     56117.0
26  37219    270     638.0      1154.0

***********************************

The Top Vaccine Numbers by Zip

       zip  cases  vaccines  population
18  37211  10233   19175.0     75955.0
0   37013  13354   18465.0     97939.0
28  37221   3910   17222.0     41017.0
12  37205   2566   14666.0     23790.0
16  37209   5072   13600.0     38664.0

***********************************

The Bottom Vaccine Numbers by Zip

       zip  cases  vaccines  population
30  37238      

###### Get Percentage of Vaccines and Covid Cases to the Population by Zip

In [66]:
# First Method of doing calculation
c_v_zip['perc_cases'] = round(c_v_zip.cases / c_v_zip.population * 100, 2)
# Second Method of doing Calculation
c_v_zip['perc_vacc'] =c_v_zip.apply(lambda row: round(100* float(row['vaccines']) / float(row['population']),2), axis=1)
# c_v_zip['perc_of_pop'] = round(c_v_zip.population / c_v_zip.population.sum() * 100,2)
# c_v_zip['perc_cases_by_pop'] = round(c_v_zip.cases / c_v_zip.population.sum() * 100,2)
# c_v_zip['perc_vaccines_by_pop'] = round(c_v_zip.vaccines / c_v_zip.population.sum() * 100,2)
c_v_zip.head()

,zip,cases,vaccines,population,perc_cases,perc_vacc
0,37013,13354,18465.0,97939.0,13.64,18.85
1,37027,231,352.0,56117.0,0.41,0.63
2,37069,16,50.0,20467.0,0.08,0.24
3,37072,2984,8460.0,31492.0,9.48,26.86
4,37076,4279,11113.0,40161.0,10.65,27.67


###### Drop the industrial Zip beltway from the records

In [67]:
c_v_zip = c_v_zip[c_v_zip['zip'] != '37213']
c_v_zip 
#confirm Line index 20 is removed. 

,zip,cases,vaccines,population,perc_cases,perc_vacc
0,37013,13354,18465.0,97939.0,13.64,18.85
1,37027,231,352.0,56117.0,0.41,0.63
2,37069,16,50.0,20467.0,0.08,0.24
3,37072,2984,8460.0,31492.0,9.48,26.86
4,37076,4279,11113.0,40161.0,10.65,27.67
5,37080,574,1970.0,7394.0,7.76,26.64
6,37115,4574,8520.0,40343.0,11.34,21.12
7,37138,2359,7152.0,23784.0,9.92,30.07
8,37189,479,1056.0,4016.0,11.93,26.29
9,37201,299,576.0,1619.0,18.47,35.58


##### Read and Upload file Hispanic or Latino Origin by Race

In [71]:
# Read the data file in Excel into the application, Skipping the header and loading sheet 0 is actual value sheet 1 = %
ethnicity_df = pd.read_excel('..\data\hispanic\originbyrace.xlsx', skiprows = 1, header=[0, 1], sheet_name = 1)
#Drop columns with Error next to each of the columns
ethnicity_df = ethnicity_df[ethnicity_df.columns.drop(list(ethnicity_df.filter(regex='Error')))]
#Drop a Level of Value headers that mean nothing to the data
ethnicity_df.columns = ethnicity_df.columns.droplevel(1)
# Remove unnecessary rows that will not be part of our statistics or you can choose to select certain rows into a DF
ethnicity_df = ethnicity_df.drop([0,1,9,10,12,13,14,15,16,17,18,19,20,21])

#Rotate the dataframe so that Zip can be column and the race data set across
ethnicity_df= ethnicity_df.set_index('Unnamed: 0_level_0').T.reset_index()
#Rename the Zip row
ethnicity_df=ethnicity_df.rename(columns={'index':'zip','Total:':'total','Two or more races:':'Two or more races',
                                         'Hispanic or Latino:':'Hispanic or Latino'})

ethnicity_df.head()

Unnamed: 0_level_0,zip,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,Two or more races,Hispanic or Latino
0,37010,0.906565,0.049335,0.000403,0.005437,0.0,0.0,0.005437,0.032823
1,37012,0.938215,0.03341,0.0,0.001373,0.0,0.0,0.003661,0.023341
2,37013,0.409234,0.339803,0.00097,0.043394,0.0,0.003819,0.031162,0.171617
3,37014,0.946032,0.006349,0.0,0.016578,0.0,0.0,0.012698,0.018342
4,37015,0.896802,0.017209,0.008322,0.006057,0.0,0.000283,0.020549,0.050778


In [72]:
# Every row starting white alone is multiplied for a Percentage representation 
#Columns 0 are skipped because of the zip code
ethnicity_df.iloc[:,1:] = ethnicity_df.iloc[:,1:] * 100
ethnicity_df.head()

Unnamed: 0_level_0,zip,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,Two or more races,Hispanic or Latino
0,37010,90.656464,4.933548,0.040274,0.543697,0.0,0.0,0.543697,3.28232
1,37012,93.82151,3.340961,0.0,0.1373,0.0,0.0,0.366133,2.334096
2,37013,40.923432,33.980335,0.096999,4.339436,0.0,0.38187,3.116225,17.161703
3,37014,94.603175,0.634921,0.0,1.657848,0.0,0.0,1.269841,1.834215
4,37015,89.680159,1.720917,0.832154,0.605718,0.0,0.028305,2.054911,5.077838
